Figure 2.

In [2]:
# sliced 3D real image
import warnings
warnings.filterwarnings("ignore")


import numpy as np
from concurrent.futures import ProcessPoolExecutor
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorly as tl

#Debugging import
import importlib
var = 'TensorDecisionTreeRegressorP' #the published version of code
package = importlib.import_module(var)
for name, value in package.__dict__.items():
    if not name.startswith("__"):
        globals()[name] = value

from TensorDecisionTreeRegressorP import *

import os
import nibabel as nib
import numpy as np
import matplotlib as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
import matplotlib.pyplot as plt

# Set parameters
np.random.seed(42)
n = 500  # first dimension of X
p1, p2 = 5, 4  # second and third dimensions of X
sigma_vals = [0, 0.01, 0.1]  # Noise levels (sigma^2 values)

# Generate the tensor X uniformly randomly from [-1, 1]
X = np.random.uniform(-1, 1, size=(n, p1, p2))

# Define the true model for y
def generate_y(X, sigma):
    epsilon = np.random.normal(0, sigma, size=n)
    y = 2 * X[:, 0, 1] * X[:, 2, 0] * X[:, 3, 0] + 3 * X[:, 1, 0] * X[:, 2, 0] * X[:, 3, 0] + epsilon
    return y

# Generate the data for different noise levels
data = {}
for sigma in sigma_vals:
    y = generate_y(X, sigma)
    data[f"sigma_{sigma}"] = y

print(np.var(data['sigma_0']))
print(np.var(data['sigma_0.01']))
print(np.var(data['sigma_0.1']))


0.43829588865781954
0.4379740908733154
0.44129308422868735


In [9]:
X_train, y_train, X_test, y_test = train_test_split(X, data['sigma_0.01'], test_size=0.3, random_state=42)
for i in range(5):
    model  =  TensorDecisionTreeRegressor(max_depth=10, min_samples_split=12,split_method='variance', split_rank=4, CP_reg_rank=3, Tucker_reg_rank=3, n_mode=4, const_array=None)
    model.fit(X_train,y_train)

    predictions = model.predict(X_train,regression_method='mean')
    print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
    predictions = model.predict(X_train,regression_method='cp')
    print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
    predictions = model.predict(X_train,regression_method='tucker')
    print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

    predictions = model.predict(X_test,regression_method='mean')
    print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
    predictions = model.predict(X_test,regression_method='cp')
    print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
    predictions = model.predict(X_test,regression_method='tucker')
    print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

TypeError: 'NoneType' object is not iterable